In [ ]:
%pip install -qqqU matplotlib

In [ ]:
from dotenv import load_dotenv

load_dotenv('../.env')

In [ ]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from langchain_aws import BedrockEmbeddings, ChatBedrock
from qdrant_client.http.models import Distance, VectorParams
from langchain_core.rate_limiters import InMemoryRateLimiter

In [ ]:
# rate_limiter = InMemoryRateLimiter(
#     requests_per_second=0.07,  # <-- Super slow! We can only make a request once every 10 seconds!!
#     check_every_n_seconds=0.1,  # Wake up every 100 ms to check whether allowed to make a request,
#     # max_bucket_size=10,  # Controls the maximum burst size.
# )

llm = ChatBedrock(
    model_id='amazon.nova-micro-v1:0',
    temperature=.0,
    region='us-east-1',
    # rate_limiter=rate_limiter
    )

embeddings = BedrockEmbeddings(region_name='us-east-1')

In [ ]:
import os

client = QdrantClient(
    location=os.environ["VECTOR_STORE_URL"],
    api_key=os.environ["VECTOR_STORE_API_KEY"]
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="regulamento-semantic",
    embedding=embeddings
)

In [ ]:
from datasets import Dataset

def pandas_to_ragas(df):
    '''
    Converts a Pandas DataFrame into a Ragas-compatible dataset
    
    Inputs:
        - df (Pandas DataFrame): The input DataFrame to be converted
        
    Returns:
        - ragas_testset (Hugging Face Dataset): A Hugging Face dataset compatible with the Ragas framework
    '''
    # Ensure all text columns are strings and handle NaN values
    text_columns = df.columns
    for col in text_columns:
        df[col] = df[col].fillna('').astype(str)
        
    # Convert 'contexts' to a list of lists
    df['reference_contexts'] = df['reference_contexts'].fillna('').astype(str).apply(eval)
    df['retrieved_contexts'] = df['retrieved_contexts'].fillna('').astype(str).apply(eval)
    
    # Converting the DataFrame to a dictionary
    data_dict = df.to_dict('list')
    
    # Loading the dictionary as a Hugging Face dataset
    ragas_testset = Dataset.from_dict(data_dict)
    
    return ragas_testset

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = """
Você escreve artigos para o regulamento  dos cursos de graduação da UFRN.

Gere um melhor palpite de artigo hipotético que responde a pergunta: {pergunta}.

Siga cuidadosamente as instruções abaixo:
- Analise cuidadosamente a pergunta e as informações necessárias para responde-la.
- Formule um ou mais artigos abrangentes que respondem diretamente pergunta.
- Não adicione nenhum conhecimento externo.
- Use um tom formal, objetivo curto e conciso.
- Os artigos devem conter uma frase e deve responder somente a pergunta.
- Escreva somente as frases artigos, sem textos de agradecimento, de apresentação do resultado ou de explicações fora do contexto.
- Não escreva um número hipotetico dos artigos, somente frases.
- Não apresente o resultado com frases do tipo: "Aqui está o resultado".
- Cada artigo deve conter um contexto único e não deve ser igual aos outros

Alguns artigos de exemplo:

A oferta do componente curricular didático-pedagógico, sob a responsabilidadedosprogramas de Pós-Graduação, deve atender, de forma sistemática e articulada, às demandasdosprogramas de Pós-Graduação da UFRN.

É facultado aos discentes regularmente matriculados nos cursos de graduaçãodaUFRN,a matrícula em componentes curriculares isolados ofertados pelos Programas de Pós-GraduaçãodaUFRN, desde que autorizado pelo coordenador do curso de graduação e do coordenador doprogramade Pós-Graduação
"""
prompt_template = PromptTemplate.from_template(prompt)

In [ ]:
import time
import random

def invoke_llm_with_backoff(llm, prompt, max_retries=5):
    retries = 0
    while retries < max_retries:
        try:
            return llm.invoke(prompt)
        except Exception as e:
            if 'Too many requests' in str(e):
                retries += 1
                wait_time = random.uniform(2 ** retries, 2 ** retries + 5)  # Exponential backoff
                print(f"Throttling error. Retrying in {wait_time:.2f} seconds...")
                time.sleep(wait_time)
            else:
                raise e
    raise Exception("Max retries reached, could not invoke model.")

In [ ]:
import pandas as pd
from tqdm import tqdm
from ragas import evaluate
from ragas.metrics import NonLLMContextRecall, NonLLMContextPrecisionWithReference

models = [
    'amazon.nova-lite-v1:0',
    'amazon.nova-micro-v1:0',
    'meta.llama3-8b-instruct-v1:0',
    'mistral.mistral-7b-instruct-v0:2',
    'mistral.mixtral-8x7b-instruct-v0:1'
    ]

all_result_df = pd.DataFrame([])

for model_name in models:
    for k in [2, 5, 10, 15, 20]:
        llm = ChatBedrock(
            model_id=model_name,
            temperature=.0,
            region='us-east-1',
            )

        df = pd.read_csv('../data/dataset_potiguana.csv')

        retrieved_contexts = []
        responses = []

        for _, row in tqdm(df.iterrows()):
            query = row['user_input']
            prompt = prompt_template.format(pergunta=query)
            hyde = invoke_llm_with_backoff(llm=llm, prompt=prompt)
            context_docs = vector_store.similarity_search(hyde.content, k=k)
            contexts = [c.page_content for c in context_docs]
            retrieved_contexts.append(contexts)

        df['retrieved_contexts'] = pd.Series(retrieved_contexts)
        eval_dataset = pandas_to_ragas(df)

        metrics = [
            NonLLMContextPrecisionWithReference(threshold=0.95),
            NonLLMContextRecall(threshold=0.95)
            ]


        results = evaluate(dataset=eval_dataset, metrics=metrics)
        result_df = results.to_pandas()
        result_df['k'] = k
        result_df['model'] = model_name
        all_result_df = pd.concat([all_result_df, result_df], ignore_index=True)

all_result_df.to_csv('../hyde_models_in_k_eval.csv')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Carregar os resultados consolidados
all_result_df = pd.read_csv("../hyde_models_in_k_eval.csv")

# Criar gráficos individuais para cada modelo
def plot_combined_metrics():
    fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharex=True)

    # Plot Context Recall
    for model_name in all_result_df['model'].unique():
        model_data = all_result_df[all_result_df['model'] == model_name]
        mean_recall = model_data.groupby('k')['non_llm_context_recall'].mean()
        axes[0].plot(mean_recall.index, mean_recall.values, marker='o', linestyle='-', label=model_name)

    axes[0].set_title('Context Recall vs. K', fontsize=16)
    axes[0].set_xlabel('K', fontsize=14)
    axes[0].set_ylabel('Context Recall', fontsize=14)
    axes[0].grid(True)
    axes[0].legend(fontsize=10)

    # Plot Context Precision
    for model_name in all_result_df['model'].unique():
        model_data = all_result_df[all_result_df['model'] == model_name]
        mean_precision = model_data.groupby('k')['non_llm_context_precision_with_reference'].mean()
        axes[1].plot(mean_precision.index, mean_precision.values, marker='s', linestyle='--', label=model_name)

    axes[1].set_title('Context Precision vs. K', fontsize=16)
    axes[1].set_xlabel('K', fontsize=14)
    axes[1].set_ylabel('Context Precision', fontsize=14)
    axes[1].grid(True)
    axes[1].legend(fontsize=10)

    plt.tight_layout()
    plt.show()

# Gerar gráficos combinados
plot_combined_metrics()


In [ ]:
import matplotlib.pyplot as plt

# Calcule a média de NonLLMContextRecall para cada valor de K
mean_recall = all_result_df.groupby('k')['non_llm_context_recall'].mean()
mean_precision = all_result_df.groupby('k')['non_llm_context_precision_with_reference'].mean()


plt.figure(figsize=(10, 6))
plt.plot(mean_recall.index, mean_recall.values, marker='o', linestyle='-', color='b', label='Mean Context Recall')
plt.plot(mean_precision.index, mean_precision.values, marker='s', linestyle='--', color='r', label='Mean Context Precision')
plt.title('Mean Context Recall and Precision vs. K using HyDE', fontsize=16)
plt.xlabel('K', fontsize=14)
plt.ylabel('Mean Score', fontsize=14)
plt.grid(True)
plt.legend(fontsize=12)

plt.xticks(mean_recall.index)  # Mostra todos os valores de K no eixo x
plt.show()

In [ ]:
models = ['amazon.nova-lite-v1:0', 'amazon.nova-micro-v1:0', 'meta.llama3-8b-instruct-v1:0', 'mistral.mistral-7b-instruct-v0:2', 'mistral.mixtral-8x7b-instruct-v0:1']

In [ ]:
from langchain_core.prompts import PromptTemplate

llm = ChatBedrock(
    model_id='mistral.mixtral-8x7b-instruct-v0:1',
    temperature=.0,
    region='us-east-1',
    )

query = 'O que preciso para fazer um estagio?'

prompt = """
Você escreve artigos para o regulamento  dos cursos de graduação da UFRN.

Gere um melhor palpite de artigo hipotético que responde a pergunta: {pergunta}.

Siga cuidadosamente as instruções abaixo:
- Analise cuidadosamente a pergunta e as informações necessárias para responde-la.
- Formule um ou mais artigos abrangentes que respondem diretamente pergunta.
- Não adicione nenhum conhecimento externo.
- Use um tom formal, objetivo curto e conciso.
- Os artigos devem conter uma frase e deve responder somente a pergunta.
- Escreva somente as frases artigos, sem textos de agradecimento, de apresentação do resultado ou de explicações fora do contexto.
- Não escreva um número hipotetico dos artigos, somente frases.
- Não apresente o resultado com frases do tipo: "Aqui está o resultado".
- Cada artigo deve conter um contexto único e não deve ser igual aos outros
- Escreva, no máximo, 3  artigos.

Alguns artigos de exemplo:

A oferta do componente curricular didático-pedagógico, sob a responsabilidadedosprogramas de Pós-Graduação, deve atender, de forma sistemática e articulada, às demandasdosprogramas de Pós-Graduação da UFRN.

É facultado aos discentes regularmente matriculados nos cursos de graduaçãodaUFRN,a matrícula em componentes curriculares isolados ofertados pelos Programas de Pós-GraduaçãodaUFRN, desde que autorizado pelo coordenador do curso de graduação e do coordenador doprogramade Pós-Graduação
"""
prompt_template = PromptTemplate.from_template(prompt)
prompt = prompt_template.format(pergunta=query)
hyde = invoke_llm_with_backoff(llm=llm, prompt=prompt)
print(hyde.content)